In [4]:
# imports
import pandas as pd

In [5]:
pd.set_option('display.width', 5000)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

In [6]:
# LOCATION DATA
location_dataset_1 = pd.read_parquet("../../Data/Location/Merged/Dataset.gzip")

# CLIENT DATA
client_dataset_1 = pd.read_csv("../../Data/Client/Data_cleaned.csv")

# POSTCODE CONVERTER
postcodes = pd.read_csv("../../Data/Location/Auxilliary/Postcode_converter.csv", sep=";")

# This dataset is rather big, so we will only use the columns we need
postcodes = postcodes[["PC6", "Buurt2022"]]

In [7]:
location_dataset_1

Dataset 2018-2022,population,WOZ,urbanity_score,d_transfer,d_mainstreet,d_supermarket,n_business,n_earners,n_owner_occupied,avg_income,household_size,k40_low_income,k20_high_income,k0_k15,k15_k25,k25_k45,k45_k65,k65+,supermarket_1km,%_single_family_homes,single_households,households_with_children,n_res_benefits,d_school
Codering,,,,,,,,,,,,,,,,,,,,,,,,
BU00140000,4480,247.0,1.0,1.3,1.7,0.4,1030.0,3800.0,12.0,22.1,1.2,59.8,10.5,75,2190,1465,465,290,9.7,13.0,2985,95,530.0,0.4
BU00140001,6630,251.0,1.0,0.8,1.3,0.3,1730.0,5500.0,12.0,22.7,1.2,61.4,9.8,115,3160,2205,670,485,7.6,9.0,4430,140,850.0,0.4
BU00140002,4150,216.0,1.0,1.6,1.4,0.5,620.0,3500.0,11.0,21.5,1.3,62.0,9.8,120,1765,1435,455,375,6.5,13.0,2615,135,670.0,0.4
BU00140003,1710,247.0,1.0,0.9,1.3,0.2,320.0,1500.0,13.0,24.8,1.3,52.6,13.6,45,685,660,150,175,7.2,11.0,1095,45,260.0,0.7
BU00140004,10,NaN,1.0,1.4,1.4,0.4,5.0,0.0,NaN,NaN,1.8,NaN,NaN,0,5,0,5,5,7.5,NaN,5,5,0.0,0.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BU19910603,480,460.0,5.0,26.2,3.2,2.7,70.0,400.0,90.0,NaN,2.7,42.7,20.5,70,70,80,150,110,0.0,84.0,30,80,120.0,2.7
BU19910604,375,500.0,5.0,25.0,2.0,2.8,75.0,300.0,84.0,NaN,2.7,41.4,19.9,60,55,60,120,80,0.0,95.0,30,65,90.0,2.7
BU19910605,365,560.0,5.0,25.3,2.3,1.9,40.0,200.0,79.0,NaN,1.9,43.9,23.8,35,40,55,155,85,0.0,95.0,105,40,70.0,2.0


In [8]:
# The aim is to add the corresponding location data to the supermarkets in the client dataset
# For every supermarket's postcode, we find the corresponding buurtcode in the postcode converter

# Retrieve the 4 digits from the postcode
client_dataset_1["Postcode prefix"] = client_dataset_1["Postcode"].apply(lambda x: str(x)[0:4])
client_dataset_1

,Supermarkt,Postcode,VVO,Huurprijs per m2,Postcode prefix
0,0,4361BH,1201.0,96.0,4361
1,1,4791HR,1300.0,143.0,4791
2,2,4101XA,1378.0,161.0,4101
3,2,3958XH,1320.0,125.0,3958
4,3,6651DE,1627.0,150.0,6651
...,...,...,...,...,...
298,1,2051GM,1385.0,188.0,2051
299,1,2201XA,1451.0,201.0,2201
300,3,7591GM,1760.0,150.0,7591
301,2,5913EG,1329.0,158.0,5913


In [9]:
def get_buurt(postcode):
    buurt = postcodes.loc[postcodes["PC6"] == postcode, "Buurt2022"]
    return buurt.values[0] if not buurt.empty else None

# Apply the function to the 'postcode' column and store the results in 'buurtcodes' column
client_dataset_1["Buurtcode"] = client_dataset_1["Postcode"].apply(get_buurt).tolist()

In [10]:
# Add '00' to the buurtcode if the length is 6, add '0' if the length is 7
client_dataset_1["BuurtcodeLength"] = client_dataset_1["Buurtcode"].apply(lambda x: len(str(x)))
client_dataset_1["Buurtcode"] = client_dataset_1["Buurtcode"].apply(lambda x: f"BU00{str(x)}" if len(str(x)) == 6 else f"BU0{str(x)}" if len(str(x)) == 7 else f"BU{x}")
client_dataset_1.set_index("Buurtcode", inplace=True)


In [11]:
client_dataset_1

,Supermarkt,Postcode,VVO,Huurprijs per m2,Postcode prefix,BuurtcodeLength
Buurtcode,,,,,,
BU07171200,0,4361BH,1201.0,96.0,4361,7
BU17090300,1,4791HR,1300.0,143.0,4791,8
BU02160008,2,4101XA,1378.0,161.0,4101,7
BU15810300,2,3958XH,1320.0,125.0,3958,8
BU02250001,3,6651DE,1627.0,150.0,6651,7
...,...,...,...,...,...,...
BU03770100,1,2051GM,1385.0,188.0,2051,7
BU05750104,1,2201XA,1451.0,201.0,2201,7
BU17741010,3,7591GM,1760.0,150.0,7591,8


In [13]:
# Store all files in a HDF5 Store file
#with pd.HDFStore("/Users/mitchlobbes/Documents/Final_Draft/Data/Model/Test_Datasets.h5", mode='w') as store:
for location_data in [("Dataset_full", location_dataset_1)]:
    total_dataset = client_dataset_1.join(location_data[1], how="left")
    
    total_dataset.drop(columns=["Postcode", "Postcode prefix", "BuurtcodeLength", "Supermarkt"], inplace=True)
    total_dataset.reset_index(inplace=True, drop=True)
    total_dataset.rename(columns={"VVO": "GLA", "Huurprijs per m2": "rental_price"}, inplace=True)
    

    order = ["rental_price", "GLA", "n_res_benefits", "population", "d_school",
                "d_transfer", "d_mainstreet", "n_business", "n_earners", "d_supermarket",
                "avg_income", "%_single_family_homes", "k20_high_income", "households_with_children",
                "single_households", "household_size", "k40_low_income", "WOZ", "n_owner_occupied",
                "urbanity_score", "supermarket_1km", "k0_k15", "k15_k25", "k25_k45", "k45_k65", "k65+"]
    
    total_dataset = total_dataset[order]

    total_dataset.to_csv("../../Data/Model/Model_data.csv", index=False)